assumes data is in `./data_movie`

In [129]:
import numpy as np
import pandas as pd
from os.path import join
import mne

The entire goal of this notebook is to fill the events and labels.

In [130]:
event_labels_dic = {
    "planetearth":0,
    "animatedshorts":1,
    "3b1b":2,
    "jackychan":3,
    "mandarinvocab":4
}

We will only load occipital lobe data. 

In [131]:
SR = 10.0 # 10 Hz sample rate
ETIME = 5.0 # defines time of event to be 5 seconds

In [132]:
def load_files(snirf_path="data_movie/steve-scan1-Sep-21-2023-6-35-PM_run02.snirf",
               tsv_path="data_movie/timestamps_run02.tsv"):
    intensity = mne.io.read_raw_snirf(snirf_path)
    annots_df = pd.read_csv(tsv_path, sep="\t")
    onsets = annots_df.Onset.tolist()
#     idx_to_type = list(set(annots_df.trial_type))
#     type_to_idx = {type_:idx for idx, type_ in enumerate(idx_to_type)}
    durations = list(np.array(onsets[1:]) - np.array(onsets[:-1])) + [0]
    annots = mne.Annotations(
        onset=onsets,  # in seconds (interprets as ctime)
        duration=durations,  # in seconds, too
        description=annots_df.trial_type.tolist(),
    )
    intensity = intensity.set_annotations(annots)
    return intensity,annots_df,onsets

In [133]:
# FRAMELEN = 128
# SR = 10.0 # 10 Hz Sample Rate
# def spec(ts,sr:float=10.0):
#     win = np.hamming(FRAMELEN)
#     slideby = FRAMELEN//4 # hard coded bad
#     nframes = len(ts)//slideby - 3
#     freqs = np.fft.rfftfreq(FRAMELEN,1/sr)
#     s=[]
#     for i in range(nframes):
#         idx_start = i*slideby
#         s.append(np.fft.rfft(ts[idx_start:idx_start+FRAMELEN] * win))
#     s = np.asarray(s)
#     times = np.arange(0,s.shape[0]) *slideby/sr
#     return s,freqs,times
# 
# s,freqs,times=spec(np.squeeze(od[0][0])[1000:-1000])

In [134]:
def plot_spec_grid(od, ch_names, tstart:float, tend:float, event_times=[0],title=None,saveas=None):
    """od is an optical density mne object
    
    event_times : list
        time of all the events, in seconds from start of file
    """
    tidx_start = int(tstart * SR)
    tidx_end = int(tend * SR)
    spec_data = [spec(np.squeeze(od[idx][0])[tidx_start:tidx_end]) for idx in range(32)]
    fig, axes = plt.subplots(nrows=8, ncols=4, figsize=(15, 25))

    for idx_ch,(ax,(s,freqs,times),ch_name) in enumerate(zip(axes.ravel(), spec_data, ch_names)):
        times += tstart # compensate for offset
        im = ax.imshow(10*np.log10(abs(s)**2), aspect='auto',cmap='plasma', vmin=-140, vmax=-70) #,vmin=-45,vmax=25)
        ax.set_xticks(np.arange(0,s.shape[1],s.shape[1]//6), [f"{i:.2f}" for i in freqs[::s.shape[1]//6]], 
                      rotation=-10, fontsize=7.0)
        ax.set_xlabel("Freqs (Hz)")
        ax.set_yticks(np.arange(0,s.shape[0],s.shape[0]//5), [f"{i:.1f}" for i in times[::s.shape[0]//5]],fontsize=7.0)
        ax.set_ylabel("Time (s)")
        ax.set_title(f"{ch_name}")
        for e in event_times:
            ax.axhline((e-tstart)*SR/(FRAMELEN//4),color='green',lw=1)
        # Colorbar for each subplot
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        
        
        # If you want the same scale for all, set vmin and vmax in imshow
        # im = ax.imshow(dat, aspect='auto', vmin=min_value, vmax=max_value)
    if title: 
        plt.suptitle(f"{title}",fontsize=20)
    plt.tight_layout()
    if saveas:
        plt.savefig(saveas,dpi=400)
    plt.show()
# plot_spec_grid(od,tstart=onsets[0],tend=onsets[-1]+20,event_times=onsets)

In [135]:
intensity,annots_df,onsets = load_files(join("data_movie","steve-scan1-Sep-21-2023-6-35-PM_run02.snirf"),
                                        join("data_movie","timestamps_run02.tsv"))
od = mne.preprocessing.nirs.optical_density(intensity)
raw_hemo = mne.preprocessing.nirs.beer_lambert_law(od)
picks = ["S2_D3 hbo", "S2_D4 hbo", "S3_D5 hbo", "S3_D6 hbo"]
_=raw_hemo.pick(picks)
raw_hemo.ch_names

Loading /Users/steve/Documents/code/neuroimagery2023/signal-classifier/data_movie/steve-scan1-Sep-21-2023-6-35-PM_run02.snirf
Reading 0 ... 13012  =      0.000 ...  1301.200 secs...


['S2_D3 hbo', 'S2_D4 hbo', 'S3_D5 hbo', 'S3_D6 hbo']

In [136]:
raw_hemo.ch_names, 

(['S2_D3 hbo', 'S2_D4 hbo', 'S3_D5 hbo', 'S3_D6 hbo'],)

In [137]:
import matplotlib.pyplot as plt

In [138]:
help(np.apply_along_axis)

Help on _ArrayFunctionDispatcher in module numpy:

apply_along_axis(func1d, axis, arr, *args, **kwargs)
    Apply a function to 1-D slices along the given axis.
    
    Execute `func1d(a, *args, **kwargs)` where `func1d` operates on 1-D arrays
    and `a` is a 1-D slice of `arr` along `axis`.
    
    This is equivalent to (but faster than) the following use of `ndindex` and
    `s_`, which sets each of ``ii``, ``jj``, and ``kk`` to a tuple of indices::
    
        Ni, Nk = a.shape[:axis], a.shape[axis+1:]
        for ii in ndindex(Ni):
            for kk in ndindex(Nk):
                f = func1d(arr[ii + s_[:,] + kk])
                Nj = f.shape
                for jj in ndindex(Nj):
                    out[ii + jj + kk] = f[jj]
    
    Equivalently, eliminating the inner loop, this can be expressed as::
    
        Ni, Nk = a.shape[:axis], a.shape[axis+1:]
        for ii in ndindex(Ni):
            for kk in ndindex(Nk):
                out[ii + s_[...,] + kk] = func1d(arr[ii +

In [139]:
def get_events_in_run(snirfpath, tsvpath):
    events = [] 
    event_labels = [] 
    intensity,annots_df,onsets = load_files(snirfpath, tsvpath)
    od = mne.preprocessing.nirs.optical_density(intensity)
    raw_hemo = mne.preprocessing.nirs.beer_lambert_law(od)
    picks = ["S2_D3 hbo", "S2_D4 hbo", "S3_D5 hbo", "S3_D6 hbo"]
    raw_hemo.pick(picks) # select only subset of channels
    # For each video
    for i in range(annots_df.shape[0]):
        onset,trial_type = float(annots_df['Onset'][i]),str(annots_df['trial_type'][i])
        if trial_type != 'rest':
            print(f"Processing trial_type {trial_type}")
            onset_next_rest = float(annots_df['Onset'][i+1])
            assert str(annots_df['trial_type'][i+1]) == 'rest' # sanity check
            # Split up video into 5second events, add events to list
            for idx in range(int(15*SR),int((onset_next_rest - onset)*SR),int(ETIME*SR)):
                e = raw_hemo[:][0][:,idx:idx+int(ETIME*SR)]
                win = np.hamming(e.shape[1])
                e = np.apply_along_axis(lambda x:x*win,1,e)
                e = abs(np.fft.rfft(e,axis=1))**2 # PSD
                e = np.log10(e) # log PSD 
                e = e[:,1:e.shape[1]//2] # drop DC and high freqs
                e += 11 # normalize ish
#                 plt.plot(e.T)
#                 plt.show()
                events.append(e)
                event_labels.append(event_labels_dic[trial_type])
    return events,event_labels

In [140]:
runids = ["run2","run3","run4-forhead","run5","run6-forhead","run7"]

snirfs = ["steve-scan1-Sep-21-2023-6-35-PM_run02.snirf",
          "steve-scan1-Sep-21-2023-7-09-PM_run03.snirf",
         "steve-scan2-Sep-21-2023-8-11-PM_run04_forhead.snirf",
         "steve-scan3-Sep-21-2023-8-35-PM_run05.snirf",
         "steve-scan4-Sep-21-2023-8-58-PM_run06_forhead.snirf",
         "steve-scan5-Sep-21-2023-9-20-PM_run07.snirf"]

tsvs = ["timestamps_run02.tsv",
        "timestamps_run03.tsv",
       "timestamps_run04_forhead.tsv",
       "timestamps_run05.tsv",
       "timestamps_run06_forhead.tsv",
       "timestamps_run07.tsv"]

events, labels, runs = [],[],[]

for runid,snirf,tsv in zip(runids,snirfs,tsvs):
    snirfpath = join("data_movie",snirf)
    tsvpath = join("data_movie",tsv)
    events_run, labels_run = get_events_in_run(snirfpath,tsvpath)
    assert len(events_run) == len(labels_run) # sanity
    print(f"Got events from {runid}, len={len(events_run)}")
    events.append(events_run)
    labels.append(labels_run)
    runs.append(runid)

Loading /Users/steve/Documents/code/neuroimagery2023/signal-classifier/data_movie/steve-scan1-Sep-21-2023-6-35-PM_run02.snirf
Reading 0 ... 13012  =      0.000 ...  1301.200 secs...
Processing trial_type planetearth
Processing trial_type animatedshorts
Processing trial_type 3b1b
Processing trial_type mandarinvocab
Got events from run2, len=205
Loading /Users/steve/Documents/code/neuroimagery2023/signal-classifier/data_movie/steve-scan1-Sep-21-2023-7-09-PM_run03.snirf
Reading 0 ... 12726  =      0.000 ...  1272.600 secs...
Processing trial_type planetearth
Processing trial_type 3b1b
Processing trial_type jackychan
Processing trial_type jackychan
Got events from run3, len=192
Loading /Users/steve/Documents/code/neuroimagery2023/signal-classifier/data_movie/steve-scan2-Sep-21-2023-8-11-PM_run04_forhead.snirf
Reading 0 ... 14487  =      0.000 ...  1448.700 secs...
Processing trial_type mandarinvocab
Processing trial_type planetearth
Processing trial_type planetearth
Processing trial_type p

In [125]:
(205+192+226+158+164+197)*ETIME/60

95.16666666666667

In [126]:
for runid,events_run,labels_run in zip(runs,events,labels):
    # serialize all events and labels 
    np.save(f"events/events_{runid}.npy",np.asarray(events_run))
    np.save(f"events/event_labels_{runid}.npy",np.asarray(labels_run))